In [ ]:
%pip install datasets

In [ ]:
import warnings
warnings.filterwarnings('ignore')

# 필수 라이브러리 임포트
import os
import torch
import numpy as np
from datetime import datetime

# Transformers 라이브러리
from transformers import (
    pipeline,                              # 고수준 API - 가장 쉬운 방법
    AutoTokenizer,                         # 자동 토크나이저
    AutoModelForQuestionAnswering,         # QA 모델 자동 로더 : 모델 패키지 
    DistilBertTokenizerFast,              # DistilBERT 고속 토크나이저
    DistilBertForQuestionAnswering,        # DistilBERT QA 모델  : 직접가져옴
    ElectraTokenizer,                      # ELECTRA 토크나이저 (한글) : 직접가져옴
    ElectraForQuestionAnswering,           # ELECTRA QA 모델 (한글) : 직접가져옴
    DefaultDataCollator,                   # 기본 데이터 콜레이터
    TrainingArguments,                     # 학습 하이퍼파라미터
    Trainer,                               # 범용 트레이너
)
from datasets import load_dataset

In [ ]:
question_answer = pipeline("question-answering",model = 'distilbert-base-cased-distilled-squad')

context = """Text mining, also referred to as text data mining (abbr.: TDM), similar to text analytics,
is the process of deriving high-quality information from text. It involves
"the discovery by computer of new, previously unknown information,
by automatically extracting information from different written resources."
Written resources may include websites, books, emails, reviews, and articles.
High-quality information is typically obtained by devising patterns and trends
by means such as statistical pattern learning. According to Hotho et al. (2005)
we can distinguish between three different perspectives of text mining:
information extraction, data mining, and a KDD (Knowledge Discovery in Databases) process."""

question1 = "What is text mining?"
question2 = "What are the perspectives of text mining?"

# 질의 응답 수행
answer1 = question_answer(context=context, question=question1)
answer2 = question_answer(context=context, question=question2)
if answer1['score'] < 0.1:
  print(f'answer1 : 답변 없음')
else:
  print(f"answer1 : {answer1['answer']}")
if answer2['score'] < 0.1:
  print(f'answer2 : 답변 없음')
else:
  print(f"answer2 : {answer2['answer']}")

# AutoModel
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-cased-distilled-squad")
model = AutoModelForQuestionAnswering.from_pretrained("distilbert-base-cased-distilled-squad")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

inputs = tokenizer(question1, context, return_tensors="pt").to(device)
with torch.no_grad():
    outputs = model(**inputs)
start_score = outputs.start_logits
end_score  = outputs.end_logits
answer_start = torch.argmax(start_score)
answer_end = torch.argmax(end_score)
answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][answer_start:answer_end+1]))
print(f"answer1 : {answer}")


Device set to use cpu


answer1 : {'score': 0.42419037222862244, 'start': 95, 'end': 153, 'answer': 'the process of deriving high-quality information from text'}
answer2 : 답변 없음


In [ ]:
#SQuAD 데이터셋 로드 분석
# 스탠포드 대학에서 공개한 질의응답 벤치마크 :
# Extractive QA(추출 기반 질의응답) 모델의 표준
# 문서 안에 이미 존재하는 답을 골라낸다.

squad = load_dataset('squad', split = 'train[:5000]', trust_remote = True)
squad = squad.train_test_split(test_size=0.2, seed=42)
squad